# Eisamay property log analysis

> **The aim of this analysis is to identify and correct the various types of errors which reach our vm machines and aim to fix them. By analysing the logs and identifying where we are facing the errors we can:**
* improve our websites response rate 
* reduce bounce rate.
* Identify key sources of errors
* Identify key problems within our code base which would be applicable across properties
* Identify platform / property / VM  specific errors 


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)

ValueError: Value must have type '<class 'int'>'

In [ ]:
# Reading the log file
df = pd.read_csv('combined_error_log.csv',index_col=False)

In [ ]:
# Seeing data
df.head(5)

> **Dropping any duplicate types of errors based on the following combination**



In [ ]:
df.drop_duplicates(subset=['PLATFORM','REQ_URL', 'REQ_FEED','DESC'], keep='last')
df.shape

In [ ]:
df.info()

* **Parsing time and date to get a more accurate analysis if needed**

In [ ]:
# Adding parsed time 
# df['TIME'] = df['TIME'].apply(lambda x: x.strip())
# df['PARSED_TIME'] =df['TIME'].apply(lambda x: x[11:].strip())
# df['PARSED_TIME'] = df['PARSED_TIME'].apply(lambda x: x[:x.rindex(':')])
# df['PARSED_DATE'] = df['TIME'].apply(lambda x: x[:11].strip())
# df['PARSED_TIME'].head(30)
# df['PARSED_TIME'] = pd.to_datetime(df['PARSED_TIME'], format='%H:%M')
# df['PARSED_TIME'].head(30)
## Parsing time and date, converting to datetime
df['PARSED_DATE_TIME'] = pd.to_datetime(df['TIME'], format='%d-%m-%Y %H:%M:%S')
df['PARSED_DATE_TIME'].dt.time.head()


hour_labels = [str(x) + '-'+str(x+1) for x in range(23)]
hour_labels.append('23-0')
hour_labels

## Binning the same to make it a categorical variable.
df["BINNED_HOURS"] = pd.cut(df.PARSED_DATE_TIME.dt.hour, bins=24,labels=hour_labels)
df['BINNED_HOURS']

* **Categorizing urls based on page types**

In [ ]:
def categorize_url(url):
    template_map = {
        'amp_':'amp',
        'articleshow':'articleshow',
        'photoshow':'photoshow',
        'photolist':'photolist',
        'videoshow':'videoshow',
        'articlelist':'articlelist',
        'topics':'topics'
    }
    for template in template_map:
        if template in url:
            return template_map[template]
    return 'others'
df['URL_CATEGORY'] = df['REQ_URL'].apply(categorize_url)


**Categorizing Errors based on Error types**

How i found the categories:
1. Initially i was assuming we generally face two types of errors: **404** for invalid feed hit, or **9000ms** where feed responds slowly. However inspecting and grouping feed errors
2. First inspected the df, found **404 and undefined**, started with these as a base along with **null**
3. Characterized others as **"others"** manually inspected others one by one and found: **410 ,403, ['object Object'] timeout and ECONNRESET**


In [ ]:
# How i found the categories:
# First inspected the df, found 404 and undefined, started with these as a base along with null
# Characterized others as "others" manually inspected others one by one and
# found: 410 ,403, ['object Object'] timeout
def categorize_error(error):
    #     categories = ['null','undefined','404','410','ERR_HTTP_HEADERS_SENT','9000ms']
    categories = {
       '[object Object]':'[object Object]','undefined':'undefined','404':'404','410':'410','403':'403','ECONNRESET':'ECONNRESET','ERR_HTTP_HEADERS_SENT':'ERR_HTTP_HEADERS_SENT','9000ms':'timeout'       
    }
    for cat in categories:
        if cat in error:
            return categories[cat]
    return 'other'
            
df['ERROR_TYPE'] = df['DESC'].apply(categorize_error)

*  **Further inspection reveals that some urls are broken with '-' '/-' or '%' values in SEO in between. It will be useful if we categorize these.**

In [ ]:
# Categories to check where we failed in urls
# Manually inspecting, i found '-/' '/-/' and cases like
df['IS_URL_ERROR'] = df['REQ_URL'].apply(lambda x: '/-/' in x or '-/' in x or '%' in x)

In [ ]:
## Some examples
df['REQ_URL'][df['IS_URL_ERROR']].head(5)

* **Analysing an error which was frequently occuring. It seems JS was appending this error without giving the actual description, need to analyse how its happening on the urls where it is occuring**

In [ ]:
# Very vague type of error
df['DESC'][df['ERROR_TYPE'] == '[object Object]']

In [ ]:
# Further inspection reveals that some urls are broken with '-' in between. It will be useful if we categorize these.
df['REQ_URL'][df['ERROR_TYPE'] == '[object Object]']

* **Following are the amp urls where the url is correctly formed, but still it fails** 

In [ ]:
df['REQ_URL'][(df['URL_CATEGORY'] == 'amp') & ~(df['IS_URL_ERROR'])].head(20)

### Helper functions for plotting

* Pie chart

In [ ]:
def plot_pie_chart(column="",labels=[],title="",percentage_dist=None):
    if not column:
        return
    plt.figure(figsize=(10,8))
    if percentage_dist:
        column_val_percentage = percentage_dist
    else:
        column_val_percentage =  df[column].value_counts(normalize=True) * 100
    if not labels:
        labels = df[column].value_counts().index
#     explode=(0,0.1)
    plt.pie(column_val_percentage,explode=explode,labels=labels, autopct='%.1f%%', 
           shadow=False, startangle=0,   
           pctdistance=1.2,labeldistance=1.4)
    plt.title(title)
    plt.show()

* Percentage vs count - barchart

In [ ]:
### Helper function to plot charts
def plot_percentage_and_count(column='',supertitle='',title_count='',title_percentage='',xlabel='',ylabel_count='COUNT',ylabel_percentage='PERCENTAGE',df=df):
    if not column:
        return
    fig,ax= plt.subplots(1,2,figsize=(16,8))
    plt.setp(ax[0].xaxis.get_majorticklabels(), rotation=90)
    plt.setp(ax[1].xaxis.get_majorticklabels(), rotation=90)

    bp = sns.barplot(data=df,x=column,y=column,orient='v',estimator=lambda x: len(x) / len(df) * 100,ax=ax[0])
    for rect in bp.patches:
        bp.text (rect.get_x() + rect.get_width() // 2,rect.get_height(),"%.2f%%"% rect.get_height(), weight='bold' )
    ax[0].set_title(title_percentage)
    ax[0].set_xlabel(xlabel)
    ax[0].set_ylabel(ylabel_percentage)

    ## Error types counts
    sns.countplot(df[column],ax=ax[1])
    ax[1].set_title(title_count)
    ax[1].set_xlabel(xlabel)
    ax[1].set_ylabel(ylabel_count)

    fig.suptitle(supertitle)

    plt.show()

* Heat map for pivots / grouped data

In [ ]:
def plot_heat_map(data=None,title="",xlabel="",ylabel="",cmap='Oranges'):
    if data is None:
        return
    plt.figure(figsize=(16,8))
    sns.heatmap(data,linewidths=.5,annot=True,cmap=cmap,fmt="d")
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

    

## Visualizing the errors

### Understanding platform level distribution for errors

In [ ]:
df['PLATFORM'].value_counts(normalize=True) * 100

In [ ]:
# Seeing platform specific
# 71 percent of errors are in desktop in this ip while 28 in mobile
# Seeing platform specific
# 71 percent of errors are in desktop in this ip while 28 in mobile
plot_pie_chart(column='PLATFORM',labels=['DESKTOP','MOBILE'],title='PLATFORM WISE ERROR DISTRIBUTIONS')

### Identifying where **the url has the problem**

In [ ]:
plot_pie_chart(column='IS_URL_ERROR',labels=['URL DIDNT HAVE ERROR','URL HAD ERROR'],title='URL ERROR DISTRIBUTIONS')


> Some examples where url was the problem

In [ ]:
df['REQ_URL'][df['IS_URL_ERROR']].head(5)

### Further categorizing the url  based  errors on  category of these urls and error types

In [ ]:
url_errors_grouped =  df[df['IS_URL_ERROR']].groupby(['URL_CATEGORY','ERROR_TYPE']).size().unstack(fill_value=0)
plot_heat_map(data=url_errors_grouped,title='URL Errors - Page and Error Types',xlabel='Error Type',ylabel='Error Type',cmap='summer')


> Above heatmap contains all urls with errors, and error types on those urls

### Understanding about feed errors wherever they occur

In [ ]:
## Seeing feed errors vs others
df[~(df['REQ_FEED'].isnull())].count()[0] / df.shape[0] 
# 41.8 percent feederrors 

In [ ]:
error_types_percentage_feeds = df[(df['ERROR_TYPE'] == '404') | (df['ERROR_TYPE'] == '410') | (df['ERROR_TYPE'] == '403') |  (df['ERROR_TYPE'] == 'timeout')] 

plot_percentage_and_count(df=error_types_percentage_feeds,column='ERROR_TYPE',xlabel="ERROR TYPES",title_count='TYPES OF ERRORS IN FEEDS - COUNT',title_percentage='TYPES OF ERRORS IN FEEDS - PERCENTAGE',supertitle='FEED TYPE  ERROR COUNTS AND PERCENTAGES')
# plot_pie_chart(percentage_dist=error_types_percentage_dist,labels=labels,title="FEED ERROR DISTRIBUTIONS")


> It seems that **majority of the feed errors are 404s**. However some feed errors are **410 and 403** respectively, which are surprising.

### Understanding distribution of errors on various templates

In [ ]:
df['URL_CATEGORY'].value_counts()

> **Articlelist + timeout issues bifurcation**

In [ ]:
plot_percentage_and_count(column='URL_CATEGORY',xlabel="TEMPLATES",title_count='TEMPLATE ERRORS - COUNT',title_percentage='TEMPLATE ERRORS - PERCENTAGE',supertitle='VARIOUS TEMPLATES ERROR COUNTS AND PERCENTAGES')

> It seems that articlelist has the highest number of errors, followed by articleshow,amp and videoshow

### Understanding distribution of various error types

In [ ]:
df['ERROR_TYPE'].value_counts()

In [ ]:
plot_percentage_and_count(column='ERROR_TYPE',xlabel="ERROR - TYPES",title_count='TYPES OF ERRORS - COUNT',title_percentage='TYPES OF ERRORS - PERCENTAGE',supertitle='TYPES OF ERRORS COUNTS AND PERCENTAGES')

### Error type vs pagetype heatmap

In [ ]:
# Creating what type of templates have what type of errors
categories_vs_error_grouped =  df.groupby(['URL_CATEGORY','ERROR_TYPE']).size().unstack(fill_value=0)
plot_heat_map(data=categories_vs_error_grouped,title='Error categories vs types',xlabel='Types of errors',ylabel='Page type',cmap='Oranges')

### Checking if time plays a factor in errors - Templates and Error Types


In [ ]:
# Creating what type of templates have what type of errors

time_vs_url_category =  df.groupby(['BINNED_HOURS','URL_CATEGORY']).size().unstack(fill_value=0)

plot_heat_map(data=time_vs_url_category,title='Error on Template vs Hour of day',xlabel='Template in which error occurred',ylabel='Hour of day',cmap='Greens')

In [ ]:
# Creating what type of templates have what type of errors
categories_vs_error_hour_of_day =  df.groupby(['BINNED_HOURS','ERROR_TYPE']).size().unstack(fill_value=0)
plot_heat_map(data=categories_vs_error_hour_of_day,title='Error categories vs Hour of day',xlabel='Types of errors',ylabel='Hour of day',cmap='summer')



> Improvements

* BOT url - classification
* ARticlelist 404s - Feed urls / Timeout ratio
* Amp pages ampnext / vs cases
* '%' url check if correctly parsed